In [1]:
from tensorflow import keras

In [5]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)
X_new = X_test[:3]


In [2]:
# make function that returns a model
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation='relu'))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [3]:
# wrap the keras model into a sklearn model
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/var/folders/rx/y661s_v17xsf84wp1l5rc16h0000gn/T/ipykernel_12921/1709004121.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)


In [8]:
# fit with early stopping, get performance on test set and predict new instances
keras_reg.fit(X_train, y_train, epochs=100,
              validation_data=(X_valid, y_valid),
              callbacks=[keras.callbacks.EarlyStopping(patience=10)])
mse_test = keras_reg.score(X_test, y_test)
y_pred = keras_reg.predict(X_new)

2023-02-21 12:16:45.207107: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2023-02-21 12:16:45.207269: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Metal device set to: Apple M1

systemMemory: 16.00 GB
maxCacheSize: 5.33 GB

Epoch 1/100


2023-02-21 12:16:45.835935: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2023-02-21 12:16:45.969443: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - ETA: 0s - loss: 1.1596

2023-02-21 12:16:48.585706: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


363/363 [==============================] - 3s 5ms/step - loss: 1.1596 - val_loss: 2.0736
Epoch 2/100
363/363 [==============================] - 2s 5ms/step - loss: 0.6304 - val_loss: 2.2717
Epoch 3/100
363/363 [==============================] - 2s 5ms/step - loss: 0.5494 - val_loss: 0.7367
Epoch 4/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4980 - val_loss: 0.5829
Epoch 5/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4666 - val_loss: 0.4195
Epoch 6/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4479 - val_loss: 0.4667
Epoch 7/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4377 - val_loss: 0.4252
Epoch 8/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4297 - val_loss: 0.4535
Epoch 9/100
363/363 [==============================] - 2s 6ms/step - loss: 0.4244 - val_loss: 0.3885
Epoch 10/100
363/363 [==============================] - 2s 5ms/step - loss: 0.4186 - val_loss: 0.4787
E

2023-02-21 12:17:50.425800: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [7]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

In [ ]:
# Sample network's hyperparameters with random search.
# Takes several hours.

param_distribs = {
    'n_hidden': [0, 1, 2, 3],
    'n_neurons': np.arange(1, 100),
    'learning_rate': reciprocal(3e-4, 3e-2)
}
rnd_search_cv = RandomizedSearchCV(keras_reg, 
                                   param_distributions=param_distribs, 
                                   n_iter=10, 
                                   cv=3)  # random search does its own splitting of trianing/validation set to determine the performance of each sampled model
rnd_search_cv.fit(X_train, y_train, 
                  epochs=100, 
                  validation_data=(X_valid, y_valid),  # only used for early stopping
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

In [ ]:
# access the results
rnd_search_cv.best_params_
rnd_search_cv.best_score_
model = rnd_search_cv.best_estimator_.model